In [28]:
import numpy as np
import pandas as pd

In [29]:
train = pd.read_csv("../data/train_preprocessed.csv")
train_x = train.drop(["target"], axis=1)
train_y = train["target"]

test_x = pd.read_csv("../data/test_preprocessed.csv")
test_x = test_x.dropna(how="any")

In [30]:
train

,age,sex,height,weight,product,amount,medical_info_a1,medical_info_a2,medical_info_a3,medical_info_b1,...,medical_keyword_6,medical_keyword_7,medical_keyword_8,medical_keyword_9,medical_keyword_10,year,month,day,yearmonth,target
0,50,1,166.445608,65.016732,9,7000000,134,202,1,11,...,1,0,1,0,0,2015,2,3,24182,0
1,68,0,164.334615,56.544217,0,7000000,438,263,3,14,...,0,1,1,0,0,2015,5,9,24185,0
2,77,1,167.462917,54.242267,2,6000000,313,325,1,18,...,1,0,1,0,0,2016,2,13,24194,1
3,17,1,177.097725,71.147762,3,8000000,342,213,2,11,...,0,0,1,0,0,2015,7,6,24187,0
4,62,0,158.165788,65.240697,1,9000000,327,102,0,14,...,0,1,1,1,0,2016,9,17,24201,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,61,1,182.729800,73.393777,1,2000000,189,232,7,17,...,0,0,1,1,0,2015,10,21,24190,0
9996,33,0,167.701136,75.006529,8,9000,426,202,3,19,...,0,0,1,1,0,2015,5,28,24185,0
9997,44,0,145.609998,47.739397,8,1000,370,274,1,11,...,0,0,1,0,1,2016,2,29,24194,0
9998,34,0,165.796017,57.567695,6,5000,291,105,1,13,...,1,1,1,1,0,2016,2,27,24194,0


In [31]:
test_x.isnull().sum()

age                   0
sex                   0
height                0
weight                0
product               0
amount                0
medical_info_a1       0
medical_info_a2       0
medical_info_a3       0
medical_info_b1       0
medical_info_b2       0
medical_info_b3       0
medical_info_c1       0
medical_info_c2       0
medical_keyword_1     0
medical_keyword_2     0
medical_keyword_3     0
medical_keyword_4     0
medical_keyword_5     0
medical_keyword_6     0
medical_keyword_7     0
medical_keyword_8     0
medical_keyword_9     0
medical_keyword_10    0
year                  0
month                 0
day                   0
yearmonth             0
dtype: int64

In [32]:
import lightgbm as lgb
from sklearn.metrics import log_loss

class Model:
    def __init__(self, params=None):
        self.model = None
        if params is None:
           self.params = {}
        else:
            self.params = params

    def fit(self, tr_x, tr_y, va_x, va_y):
        params = {'objective': 'binary', 'seed': 71, 'verbose': 0, 'metrics': 'binary_logloss'}
        num_round = 100
        params.update(self.params)

        dtrain = lgb.Dataset(tr_x, tr_y)
        dvalid = lgb.Dataset(va_x, va_y)

        self.model = lgb.train(
            params, dtrain, num_boost_round=num_round, valid_names=["train", "valid"], valid_sets=[dtrain, dvalid]
        )
    def perdict(self, x):
        pred = self.model.predict(x)
        return pred


In [33]:
# hold-out法での分割
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
tr_x, va_x, tr_y, va_y = train_test_split(train_x, train_y, test_size=0.25, random_state=71, shuffle=True)

In [34]:
model = Model()
model.fit(tr_x, tr_y, va_x, va_y)

[LightGBM] [Warning] Starting from the 2.1.2 version, default value for the "boost_from_average" parameter in "binary" objective is true.
This may cause significantly different results comparing to the previous versions of LightGBM.
Try to set boost_from_average=false, if your old models produce bad results
[1]	train's binary_logloss: 0.454308	valid's binary_logloss: 0.465515
[2]	train's binary_logloss: 0.429565	valid's binary_logloss: 0.443444
[3]	train's binary_logloss: 0.410077	valid's binary_logloss: 0.425543
[4]	train's binary_logloss: 0.39358	valid's binary_logloss: 0.410625
[5]	train's binary_logloss: 0.379354	valid's binary_logloss: 0.397666
[6]	train's binary_logloss: 0.365913	valid's binary_logloss: 0.387422
[7]	train's binary_logloss: 0.354309	valid's binary_logloss: 0.376037
[8]	train's binary_logloss: 0.344354	valid's binary_logloss: 0.366734
[9]	train's binary_logloss: 0.334834	valid's binary_logloss: 0.35898
[10]	train's binary_logloss: 0.326209	valid's binary_logloss: 0

In [37]:
va_pred = model.perdict(va_x)
va_pred

array([0.00738854, 0.38913896, 0.00381021, ..., 0.1096412 , 0.08484495,
       0.00803791])

In [40]:
score = log_loss(va_y, va_pred)
score

0.21726392718295187

In [43]:
# cross validation
from sklearn.model_selection import KFold
kf = KFold(n_splits=4, shuffle=True, random_state=71)

scores = []
for tr_idx, va_idx, in kf.split(train_x):
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

    model = Model()
    model.fit(tr_x, tr_y, va_x, va_y)
    va_pred = model.perdict(va_x)
    score = log_loss(va_y, va_pred)
    scores.append(score)

np.mean(scores)

[LightGBM] [Warning] Starting from the 2.1.2 version, default value for the "boost_from_average" parameter in "binary" objective is true.
This may cause significantly different results comparing to the previous versions of LightGBM.
Try to set boost_from_average=false, if your old models produce bad results
[1]	train's binary_logloss: 0.454308	valid's binary_logloss: 0.465515
[2]	train's binary_logloss: 0.429565	valid's binary_logloss: 0.443444
[3]	train's binary_logloss: 0.410077	valid's binary_logloss: 0.425543
[4]	train's binary_logloss: 0.39358	valid's binary_logloss: 0.410625
[5]	train's binary_logloss: 0.379354	valid's binary_logloss: 0.397666
[6]	train's binary_logloss: 0.365913	valid's binary_logloss: 0.387422
[7]	train's binary_logloss: 0.354309	valid's binary_logloss: 0.376037
[8]	train's binary_logloss: 0.344354	valid's binary_logloss: 0.366734
[9]	train's binary_logloss: 0.334834	valid's binary_logloss: 0.35898
[10]	train's binary_logloss: 0.326209	valid's binary_logloss: 0

0.21597230558462338

In [44]:
# -----------------------------------
# Stratified K-Fold
# -----------------------------------
from sklearn.model_selection import StratifiedKFold
kf = StratifiedKFold(n_splits=4, shuffle=True, random_state=71)

for tr_idx, va_idx in kf.split(train_x, train_y):
    print(tr_idx.shape)
    print(va_idx.shape)

(7500,)
(2500,)
(7500,)
(2500,)
(7500,)
(2500,)
(7500,)
(2500,)


In [45]:
# GroupKfold
train_x["user_id"] = np.arange(0, len(train_x)) // 4

In [46]:
train_x["user_id"]

0          0
1          0
2          0
3          0
4          1
        ... 
9995    2498
9996    2499
9997    2499
9998    2499
9999    2499
Name: user_id, Length: 10000, dtype: int64

In [48]:
from sklearn.model_selection import  KFold, GroupKFold
user_id = train_x['user_id']
unique_user_ids = user_id.unique()
unique_user_ids.shape

(2500,)

In [56]:
scores = []
kf = KFold(n_splits=4, shuffle=True, random_state=71)
for tr_group_idx, va_group_idx in kf.split(unique_user_ids):
    tr_groups, va_groups = unique_user_ids[tr_group_idx], unique_user_ids[va_group_idx]

    print(tr_groups.shape, va_groups.shape)

    is_tr = user_id.isin(tr_groups)
    is_va = user_id.isin(va_groups)

    tr_x,va_x = train_x[is_tr], train_x[is_va]
    print(tr_x.shape, va_x.shape)


(1875,) (625,)
(7500, 29) (2500, 29)
(1875,) (625,)
(7500, 29) (2500, 29)
(1875,) (625,)
(7500, 29) (2500, 29)
(1875,) (625,)
(7500, 29) (2500, 29)
